In [23]:
import numpy as np
import torch
from torch.utils import data

true_w = torch.tensor([2,-3.4])
true_b = 4.2

同理的人造数据生成函数

In [24]:
def synthetic_data(w,b,num_examples):
    X = torch.normal(0,1,(num_examples,len(w)))#行数为样本数，列数为w的行数的矩阵
    y = torch.matmul(X,w) + b#做X矩阵（二维张量）和w向量（一维张量）的乘法，然后加上b，这里根据tensor的运算特性，b虽然只是一个值，但是会加到算出的一维张量的每一个元素上
    y += torch.normal(0,0.01,y.shape)#加上噪声
    return X,y.reshape((-1,1))#把y重新变成列向量，这里是二维张量，多行一列

features, labels = synthetic_data(true_w,true_b,1000)

用torch自带的data.TensorDataset类（也就是张量数据集类）以及自带的data.DataLoader迭代器来构造并输出一个数据集迭代器，每次迭代输出一个小批次的数据集

In [25]:
def load_array(data_arrays,batch_size,is_train=True):
    '''构造并输出一个数据迭代器'''
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

batch_size = 10
data_iter = load_array((features,labels),batch_size)

next(iter(data_iter))

[tensor([[-1.2630, -1.7926],
         [-0.7948, -0.4161],
         [ 0.6755,  0.9307],
         [ 1.2700,  0.1382],
         [-0.9338, -1.1272],
         [-0.5846,  0.0662],
         [ 0.3538,  0.3069],
         [ 0.7527,  1.9665],
         [-1.8902, -0.8176],
         [ 2.5099,  0.4185]]),
 tensor([[ 7.7929],
         [ 4.0276],
         [ 2.3925],
         [ 6.2793],
         [ 6.1634],
         [ 2.8137],
         [ 3.8615],
         [-0.9818],
         [ 3.2019],
         [ 7.7827]])]

定义线性模型，这里直接用torch自带的线性模型nn.Linear

In [26]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

初始化模型参数（这里的\[0\]）是因为上一段中用了nn.Sequential意思是把这个模型放到一个模型序列中，所以调用要调用第0层

In [27]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

用模型自己定义好的损失函数，和我们之前写的损失函数没什么区别，只是它已经求过和了

In [28]:
loss = nn.MSELoss()

从torch自带的优化算法中选择SGD算法

In [29]:
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

训练过程，和上一个没什么区别，只是这次用torch自带的工具封装了

In [30]:
num_epochs = 3
for epoch in range(num_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch + 1},loss {l.detach():f}')

epoch 1,loss 0.000254
epoch 2,loss 0.000109
epoch 3,loss 0.000110
